## CAPM & FRENCH FAMA ANALYSIS

# 1. Import libraries

In [ ]:
!pip install pandas numpy matplotlib statsmodels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# 2. Load Data

In [ ]:
# swedish companies data loading
astrazeneca_data = pd.read_csv('Swedish_companies_251030_AH_AstraZeneca.csv')
axfood_data = pd.read_csv('Swedish_companies_251030_AH_Axfood.csv')
bonava_data = pd.read_csv('Swedish_companies_251030_AH_Bonava.csv')
government_bonds_data = pd.read_csv('Swedish_companies_251030_AH_Government_Bond_10_year.csv')
hufvudstaden_data = pd.read_csv('Swedish_companies_251030_AH_Hufvudstaden.csv')
sbb_data = pd.read_csv('Swedish_companies_251030_AH_SBB.csv')
sixrx_data = pd.read_csv('Swedish_companies_251030_AH_SIXRX.csv')

# US companies data loading
apple_data = pd.read_csv('US_companies_251030_AH_Apple.csv')
cocacola_data = pd.read_csv('US_companies_251030_AH_CocaCola.csv')
ff_data = pd.read_csv('US_companies_251030_AH_FF.csv')
microsoft_data = pd.read_csv('US_companies_251030_AH_Microsoft.csv')

# 3. Convert swedish risk free rate to weekly

In [ ]:
rf_annual = government_bonds_data[['Annual_Yield']].copy()
rf_annual.columns = ['RF_Annual']
rf_weekly = (rf_annual / 100) * (7 / 365)
rf_weekly = rf_weekly.squeeze()  # Convert to Series

print(rf_weekly.head())
print(type(rf_weekly))


# 4. Compute weekly return for swedish stocks

In [ ]:
def compute_returns(df):
    return df.pct_change().dropna()

returns_sweden = compute_returns(astrazeneca_data['Adjusted_Close_Price']).to_frame(name='AstraZeneca')
returns_sweden['Axfood'] = compute_returns(axfood_data['Adjusted_Close_Price'])
returns_sweden['Bonava'] = compute_returns(bonava_data['Adjusted_Close_Price'])
# returns_sweden['Government Bonds'] = compute_returns(government_bonds_data['Annual_Yield'])
returns_sweden['Hufvudstaden'] = compute_returns(hufvudstaden_data['Adjusted_Close_Price'])
returns_sweden['SBB'] = compute_returns(sbb_data['Adjusted_Close_Price'])
returns_sweden['SIXRX'] = compute_returns(sixrx_data['Close'])

# returns_us = compute_returns(apple_data['Adjusted_Close_Price']).to_frame(name='Apple')
# returns_us['CocaCola'] = compute_returns(cocacola_data['Adjusted_Close_Price'])
# # returns_us['FF'] = compute_returns(ff_data['Adjusted_Close_Price'])
# returns_us['Microsoft'] = compute_returns(microsoft_data['Adjusted_Close_Price'])  

print(returns_sweden.head())
# print(returns_us.head())

# 5. CAPM regression for each swedish stocks

In [ ]:
# def run_capm(stock_ret, market_ret, rf):
#     y = stock_ret - rf
#     X = market_ret - rf
#     X = sm.add_constant(X)
#     model = sm.OLS(y, X).fit()
#     return model

# model = run_capm(returns_sweden['AstraZeneca'], returns_sweden['SIXRX'], rf_weekly)
# model.summary()

# def run_capm(stock_ret, market_ret, rf):
#     common_index = stock_ret.index.intersection(market_ret.index).intersection(rf.index)
#     y = stock_ret.loc[common_index] - rf.loc[common_index]
#     X = market_ret.loc[common_index] - rf.loc[common_index]
#     X = sm.add_constant(X)
#     model = sm.OLS(y, X).fit()
#     return model

# model = run_capm(returns_sweden['AstraZeneca'], returns_sweden['SIXRX'], rf_weekly)
# model.summary()

def run_capm(stock_ret, market_ret, rf):
    common_index = stock_ret.index.intersection(market_ret.index).intersection(rf.index)
    y = stock_ret.loc[common_index] - rf.loc[common_index]
    X = market_ret.loc[common_index] - rf.loc[common_index]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model

model = run_capm(returns_sweden['AstraZeneca'], returns_sweden['SIXRX'], rf_weekly)
model.summary()



# 6. Create Equally Weighted portfolio

In [ ]:
def equal_weight_portfolio(df):
    return df.mean(axis=1)


portfolio = equal_weight_portfolio(returns_sweden[['AstraZeneca','Axfood','Bonava','Hufvudstaden','SIXRX','SBB']])
print(portfolio.head())

# 7. Systematic VS Unsystematic error

In [ ]:
def risk_decomposition(stock_ret, market_ret, beta):
    sigma_m = np.var(market_ret)
    systematic = (beta**2) * sigma_m
    unsystematic = np.var(stock_ret) - systematic
    return systematic, unsystematic

In [ ]:
# Extract beta from the CAPM regression model
beta = model.params.iloc[1]  # The slope coefficient is beta

# Calculate systematic and unsystematic risk
systematic_risk, unsystematic_risk = risk_decomposition(
    returns_sweden['AstraZeneca'], 
    returns_sweden['SIXRX'], 
    beta
)

print(f"Beta: {beta:.6f}")
print(f"Systematic Risk: {systematic_risk:.6f}")
print(f"Unsystematic Risk: {unsystematic_risk:.6f}")
print(f"Total Risk: {systematic_risk + unsystematic_risk:.6f}")


## 8. Fama-French Three-Factor Regression (US Data)


In [ ]:
def run_ff3(stock_ret, rf, mkt, smb, hml):
    # Find common index across all series
    common_index = stock_ret.index.intersection(rf.index).intersection(mkt.index).intersection(smb.index).intersection(hml.index)
    
    # Align all data to common index
    stock_ret = stock_ret.loc[common_index]
    rf = rf.loc[common_index]
    mkt = mkt.loc[common_index]
    smb = smb.loc[common_index]
    hml = hml.loc[common_index]
    
    # Run regression
    y = stock_ret - rf
    X = pd.concat([mkt - rf, smb, hml], axis=1)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model

# Run Fama-French 3-Factor model for all three US companies
print("=" * 80)
print("FAMA-FRENCH 3-FACTOR REGRESSION RESULTS")
print("=" * 80)

# Apple
print("\n" + "=" * 80)
print("APPLE (AAPL)")
print("=" * 80)
ff3_apple = run_ff3(apple_data['Adjusted_Close_Price'], ff_data['RF'], ff_data['Mkt-RF'], ff_data['SMB'], ff_data['HML'])
print(ff3_apple.summary())

# Microsoft
print("\n" + "=" * 80)
print("MICROSOFT (MSFT)")
print("=" * 80)
ff3_microsoft = run_ff3(microsoft_data['Adjusted_Close_Price'], ff_data['RF'], ff_data['Mkt-RF'], ff_data['SMB'], ff_data['HML'])
print(ff3_microsoft.summary())

# Coca-Cola
print("\n" + "=" * 80)
print("COCA-COLA (KO)")
print("=" * 80)
ff3_cocacola = run_ff3(cocacola_data['Adjusted_Close_Price'], ff_data['RF'], ff_data['Mkt-RF'], ff_data['SMB'], ff_data['HML'])
print(ff3_cocacola.summary())


In [ ]:
# Summary comparison of all three companies
print("\n" + "=" * 100)
print("FAMA-FRENCH 3-FACTOR MODEL COMPARISON - ALL THREE COMPANIES")
print("=" * 100)

companies = {
    'Apple (AAPL)': ff3_apple,
    'Microsoft (MSFT)': ff3_microsoft,
    'Coca-Cola (KO)': ff3_cocacola
}

for company_name, model in companies.items():
    print(f"\n{company_name}:")
    print(f"  Alpha (Intercept):        {model.params.iloc[0]:8.6f}  (p-value: {model.pvalues.iloc[0]:.4f})")
    print(f"  Market Risk Premium Beta: {model.params.iloc[1]:8.6f}  (p-value: {model.pvalues.iloc[1]:.4f})")
    print(f"  SMB (Size) Beta:          {model.params.iloc[2]:8.6f}  (p-value: {model.pvalues.iloc[2]:.4f})")
    print(f"  HML (Value) Beta:         {model.params.iloc[3]:8.6f}  (p-value: {model.pvalues.iloc[3]:.4f})")
    print(f"  R-squared:                {model.rsquared:8.6f}")
    print(f"  Adj. R-squared:           {model.rsquared_adj:8.6f}")


## 9. Plotting CAPM Scatter with Trendline


In [ ]:

def capm_plot(stock_excess, market_excess):
    plt.scatter(market_excess, stock_excess)
    m, b = np.polyfit(market_excess, stock_excess, 1)
    plt.plot(market_excess, m*market_excess + b)
    plt.xlabel('Market Excess Return')
    plt.ylabel('Stock Excess Return')
    plt.title('CAPM Regression Line')
    plt.show()

capm_plot(returns_sweden['AstraZeneca'] - rf_weekly, returns_sweden['SIXRX'] - rf_weekly)